In [1]:
import pandas as pd
import numpy as np
import random
import re
import torch
from tqdm import tqdm
import transformers

In [2]:
from transformers import AutoTokenizer, AutoModel

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased').to(device)

In [ ]:
df = pd.read_csv("KG_translation_triplets_results.csv")
df.dropna(inplace = True)
df

In [ ]:
triplets = (df['sub'] + " " + df['pred'] + " " + df['obj']).tolist()

In [ ]:
sub_obj = (df['sub'] + " " + df['obj']).tolist()
predicate = df['pred'].tolist()

In [ ]:
weights = []
for pred, sub in tqdm(zip(predicate, sub_obj), total = len(predicate)):
    input_ids = torch.tensor(tokenizer.encode(pred)).unsqueeze(0).to(device)
    output1 = model(input_ids)[0][0][0]
    input_ids = torch.tensor(tokenizer.encode(sub)).unsqueeze(0).to(device)
    output2 = model(input_ids)[0][0][0]
    outputs = torch.dot(output1, output2)/(torch.linalg.norm(output1) * torch.linalg.norm(output2))
    weights.append(outputs.item())

In [ ]:
df['pred_weights'] = weights

In [ ]:
df.to_csv("KG_translation_triplets_results.csv")